<a href="https://colab.research.google.com/github/LorenzoJoquino/AP157/blob/main/GDELT_Short_Project_1_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This part of the program aims to read, analyze, and process GDELT files

In [205]:
#start date as 20220315020434, end date as 20220315033434

In [206]:
#Import necessary libraries 
from tabulate import tabulate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import networkx as nx
import itertools 
from google.colab import files
from google.colab import drive
import gdown 
import requests 
from pathlib import Path
import random as rnd
import re

In [207]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dictionary of Some Variables and Functions

###Variables

In [208]:
# "The Dictionary"

GDELT_types = ["export", "mentions"] #will add gkg later on

url_header = 'http://data.gdeltproject.org/gdeltv2/'
root_directory = [a + b + c for a, b, c in zip(["/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_"]*len(GDELT_types) \
                                               , GDELT_types, ["/"]*len(GDELT_types))]

filename_tail = [a + b + c for a, b, c in zip(["."]*len(GDELT_types) , GDELT_types, [".CSV.zip"]*len(GDELT_types))]

GDELT_v2_export_id = "1qu21fjfQIZ3ut7ozOyBUs7pXngQx_pR6" #folder id of GDELT_v2_export 
GDELT_v2_mentions_id = "1pfIL16rrkMcxqUoNaGPp2d3PKendiz2b" #folder id of GDELT_v2_mentions 

interval_min = 15 #minute intervals of GDELT 
date_format = "YYYYmmddHHMMSS"
len_date_format = len(date_format)

###Functions for GDELT Headers 

In [209]:
# Function to obtain GDELT headers 
root_directory_headers = "/content/drive/MyDrive/GDELT_Raw_data/column_headers/"
column_headers_filename = ["GDELT_export_ColNames.txt", "GDELT_mentions_ColNames.txt"]

def give_GDELT_mentions_ColNames():
  file = open(root_directory_headers + column_headers_filename[1], "r")
  ColNames_file = file.read()
  ColNames = ColNames_file.split("\uf0b7")
  #the weird question mark has a character assignment \uf0b7
  #column_header_file.close()
  for i in range(len(ColNames)):
    split_string = ColNames[i].split(") ", 1)
    ColNames[i] = "*" + split_string[0] + "*"

  return ColNames

def give_GDELT_export_ColNames():
  file = open(root_directory_headers + column_headers_filename[0], "r")
  ColNames_file = file.read()
  ColNames = ColNames_file.split("\n")
  #new line delimiter as \n
  #column_header_file.close()
  for i in range(len(ColNames)):
    split_string = ColNames[i].split(",N", 1)
    ColNames[i] = split_string[0] 

  return ColNames

column_headers = {"export": give_GDELT_export_ColNames(), "mentions": give_GDELT_mentions_ColNames()}

###Function to obtain a date range (get_date_range)

In [210]:
# Function to obtain a date range 
def get_date_range():

  def ask_for_terminal_dates():
    #asking for input for terminal dates 
    try:
      start_date_obj = input("Input datetime you want to start at? (Format: YYYYmmddHHMMSS) ")
      if len(start_date_obj) != len_date_format: start_date_obj = 'error'
      start_date = pd.to_datetime(start_date_obj, format='%Y%m%d%H%M%S')

    except:
      start_date = dt.datetime.today() - dt.timedelta(days=3)

    start_date = start_date.replace(microsecond=0)
    print("%s is your start date" %start_date) #just for checking 
    
    try:
      end_date_obj = input("Input datetime you want to end at? (Format: YYYYmmddHHMMSS) ")
      if len(end_date_obj) != len_date_format: end_date_obj = 'error'  #not sure if to make it an error or to just add zeros 
      end_date = pd.to_datetime(end_date_obj, format='%Y%m%d%H%M%S')
    except: 
      end_date = dt.datetime.today() - dt.timedelta(days=2)
    end_date = end_date.replace(microsecond=0)
    print("%s is your end date" %end_date) #just for checking 
    return start_date, end_date

  def create_date_range(start_date, end_date):
    #must round down end_date and round up start_date to the nearest 15-minute interval 
    end_date_time = end_date.time()
    end_date = end_date - dt.timedelta(seconds = end_date_time.second, minutes = end_date_time.minute % interval_min)
    print("%s is your new end date" %end_date)

    start_date_time = start_date.time()
    start_date = start_date - dt.timedelta(seconds = start_date_time.second, minutes = start_date_time.minute % interval_min) #will adjust this later 
    print("%s is your new start date" %start_date)

    #make a list of dates in between start_date and end_date 
    datetimes_range = []
    scanning_date = start_date 
    while(scanning_date < end_date):
      # print(scanning_date)
      datetimes_range.append(scanning_date)
      scanning_date = scanning_date + dt.timedelta(minutes = interval_min)
    datetimes_range.append(end_date)

    return datetimes_range 


  #must string the values of the date together to form it back to a number 
  def convert_date_to_number(datetimes_range): 
    datetimes_range_int = []
    for date_in_dtformat in datetimes_range:
      date_in_intformat = int(date_in_dtformat.strftime("%Y%m%d%H%M%S"))
      datetimes_range_int.append(date_in_intformat)
    
    return datetimes_range_int 

  #now run all the subfunctions:
  start_date, end_date = ask_for_terminal_dates()
  datetimes_range = create_date_range(start_date, end_date)
  datetimes_range_int = convert_date_to_number(datetimes_range)

  return datetimes_range_int 

###Function to create Filepaths of any type (export/event, mention) from data range (get_filepaths(dates_to_read))

Next code is to create filepaths so that it is possible to read files from GDRIVE once they are downloaded and saved in there 

In [211]:
#create all the GDRIVE paths of each filename 
def get_filepaths(dates_to_read):

  def create_filepath(date, type_num):
    filepath = root_directory[type_num] + str(date) + filename_tail[type_num]
    return filepath

  filepaths_any_type = []
  for GDELT_type_num in range(len(GDELT_types)):
    filepaths_type_specific = []
    for date in dates_to_read:
      filepath = create_filepath(date, GDELT_type_num)
      filepaths_type_specific.append(filepath)
    
    filepaths_any_type.append(filepaths_type_specific)

  return filepaths_any_type 

## Reading and Storing Export(Event) File

Input: non, but implicitly a date range; Output: list of event dfs, list of filenames that were read 

In [212]:
def read_export_file(dates_to_read):
  #dates_to_read = get_date_range()
  filepaths = get_filepaths(dates_to_read)

  df_export_aggregate = []
  exports_that_were_read = []
  for file in filepaths[0]: #0 for export files
    try: 
      df = pd.read_csv(file, delimiter='\t', header=None, names=column_headers["export"])
      pd.options.display.max_columns = 60
      #print(df)
      df_export_aggregate.append(df)
      exports_that_were_read.append(file)

    except:
      print("no file of " + file + " .")
      continue
  return df_export_aggregate, exports_that_were_read #returns list of df's of export files 

## Reading and storing Mentions File

Input: non, but implicitly a date range; Output: list of mention dfs, list of filenames that were read 

In [213]:
def read_mentions_file(dates_to_read):
  #dates_to_read = get_date_range()
  filepaths = get_filepaths(dates_to_read)

  df_mentions_aggregate = []
  mentions_that_were_read = []
  for file in filepaths[1]:
    try: 
      df = pd.read_csv(file, delimiter='\t', header=None, names=column_headers["mentions"])
      pd.options.display.max_columns = 60
      #print(df)
      df_mentions_aggregate.append(df)
      mentions_that_were_read.append(file)
    except:
      print("no file of " + file + " .")
      continue
  return df_mentions_aggregate, mentions_that_were_read #returns list of df's

##Get list of all Event IDs 

Input:Aggregate of dfs of event files, Output: List of all Event IDs and their date-added

In [214]:
def get_eventIDs(eventdf_agg): 
  eventIDs = []
  eventID_dates = []

  for eventdf in eventdf_agg:
    eventIDs.append(eventdf['GLOBALEVENTID,INTEGER'].tolist())
    eventID_dates.append(eventdf['DATEADDED,INTEGER'].tolist()) #get date event was added so that I can compare it to the date of the mention dfs

  #converts from a list of lists to a single list  
  eventIDs_singlelist = list(itertools.chain.from_iterable(eventIDs))  
  eventID_dates_singlelist = list(itertools.chain.from_iterable(eventID_dates)) 
  
  return eventIDs_singlelist, eventID_dates_singlelist

##Get list of all URLs that mention/refer to a certain event (event ID) 

Input: event ID_with date, list of mention files; Output: List of URLs
- Input is in form [eventID, date_Added]  

In [215]:
def get_URLs_mentioning_event(eventID_withdate, mentiondf_agg):
  URLs_mentioning_event = []

  for mentiondf in mentiondf_agg:
    
    #code to constrain the mentiondfs being scanned 
    date_mentiondf = mentiondf.iloc[200]['* MentionTimeDate. (integer*'] #all the date added in the mentions should be the same, so any row number will do 

    if (date_mentiondf < eventID_withdate[1]): #if the date of the mentiondf is earlier than when the event happened (earlier dates will have a lower number)
      continue #skip the mentiondf, for sure there will be no mentions of that event 

    #code to filter for URLs 
    mentions_of_eventID = mentiondf.loc[mentiondf['*GlobalEventID. (integer*'] == eventID_withdate[0]]
    #print(mentions_of_eventID) #for checking
    URLs_mentioning_event.append(mentions_of_eventID['* MentionIdentifier. (integer*'].tolist())
  
  URLs_mentioning_event_singlelist = list(itertools.chain.from_iterable(URLs_mentioning_event)) #convert from list of lists of URLs to single list of URLs

  return URLs_mentioning_event_singlelist 

##Get common URLs between two event IDs (co-mentioning) 

1. Input: event 1 and event 2 ; Output: intersection? or number of common URLs? 


In [216]:
def get_common_URLs_andFreq(event_withdate1, event_withdate2, mentiondf_agg):

  #First getting list of URLs
  URLs_event1 = get_URLs_mentioning_event(event_withdate1, mentiondf_agg)
  URLs_event2 = get_URLs_mentioning_event(event_withdate2, mentiondf_agg)

  print(len(URLs_event1), len(URLs_event2))

  if len(URLs_event1) == 1: 
    if URLs_event1[0] in URLs_event2:
      return 1
    else:
      return 0
  else: 
    #Now getting the common URLs
    URLs_event1_as_set = set(URLs_event1)
    Intersection = URLs_event1_as_set.intersection(URLs_event2)
    Intersection_as_list = list(Intersection)
    
  return len(Intersection_as_list)

##Create list of eventIDs and their comentioning between them 

Input: List of all events; Output: List([event1, event2, number of comentions]) 

###First Option: Get URLs as you pair them Up

In [217]:
def create_CoMention_list(dates_to_read):
  df_export_agg, scanned_export_filenames = read_export_file(dates_to_read)    #this is your event df 
  df_mentions_agg, scanned_mention_filenames = read_mentions_file(dates_to_read)

  eventIDs_withdates_listed = np.array(get_eventIDs(df_export_agg)) #Format: [[events],[dates]]
  eventIDs_withdates_paired = eventIDs_withdates_listed.transpose() #Format: [[event1, date1],....,[eventN, dateN]]

  print("finished getting all event IDs")

  CoMentioning_list = []
  scanned_event_pairs = []
  counter = 0 #Just to trace where it's at in terms of pairs scanned

  for event_pair in itertools.combinations(eventIDs_withdates_paired, 2):
    event_pair = list(event_pair)
    
    #print(event_pair) #for checking
    print("Pairs scanned: " + str(counter))

    counter += 1

    #I figured that if I'm going to use itertools.combinations, I don't need these if-else statements 
    """if (event_pair in scanned_event_pairs).all():
      continue
    elif (event_pair[0] == event_pair[1]).all(): #need to add .all() since numpy array being used
      continue
    
    else: 
      num_common_URLs = get_common_URLs_andFreq(event_pair[0], event_pair[1], df_mentions_agg)

      print([event_pair[0][0], event_pair[1][0], num_common_URLs]) #for checking

      CoMentioning_list.append([event_pair[0][0], event_pair[1][0], num_common_URLs]) 
      scanned_event_pairs.append(list(event_pair))
      scanned_event_pairs.append([event_pair[1], event_pair[0]]) #reverse of list(event_pair)"""

    num_common_URLs = get_common_URLs_andFreq(event_pair[0], event_pair[1], df_mentions_agg)

    print([event_pair[0][0], event_pair[1][0], num_common_URLs]) #for checking

    CoMentioning_list.append([event_pair[0][0], event_pair[1][0], num_common_URLs]) 
    scanned_event_pairs.append(list(event_pair))
    scanned_event_pairs.append([event_pair[1], event_pair[0]]) #reverse of list(event_pair)
    
  return CoMentioning_list

###Second Option: Get All URLs first before pairing them up 

##Create Network from the list of comentions 

Input: List of Comentions; Output: Network (nodes --> event ID, edges --> # of comentions)

In [218]:
def create_network(list_of_list):
  co_mention_df = pd.DataFrame(list_of_list, columns = ['Event_1', 'Event_2', 'Num_Co_mentions']) #convert first to dataframe 

  #convert to network and draw 
  plt.figure()
  co_mention_network = nx.from_pandas_edgelist(co_mention_df, source = 'Event_1', target = 'Event_2', edge_attr='Num_co_mentions')
  nx.draw(co_mention_network)
  plt.title("GDELT co-mention Grid")
  plt.draw()  # pyplot draw()`
  
  #get adjacency matrix
  Adj_co_mention = nx.to_numpy_array(co_mention_network)
  return Adj_co_mention

##Put Everything together from reading dfs to creating networks

1. Get Date Range 
2. create_CoMention_list
3. create network 

In [219]:
#start date as 20220315020434, end date as 20220315033434

In [220]:
def main_networks():
  dates_to_read = get_date_range()
  print("finished with getting dates")
  CoMentioning_list = create_CoMention_list(dates_to_read)
  Adj_co_mention = create_network(CoMentioning_list)

  print(Adj_co_mention)
  return 



###Actual Test

In [221]:
main_networks()

Streaming output truncated to the last 5000 lines.
[1033864016, 1033869465, 0]
Pairs scanned: 66969
1 1
[1033864016, 1033869466, 0]
Pairs scanned: 66970
1 1
[1033864016, 1033869467, 0]
Pairs scanned: 66971
1 1
[1033864016, 1033869468, 0]
Pairs scanned: 66972
1 1
[1033864016, 1033869469, 0]
Pairs scanned: 66973
1 1
[1033864016, 1033869470, 0]
Pairs scanned: 66974
1 1
[1033864016, 1033869471, 0]
Pairs scanned: 66975
1 1
[1033864016, 1033869472, 0]
Pairs scanned: 66976
1 1
[1033864016, 1033869473, 0]
Pairs scanned: 66977
1 1
[1033864016, 1033869474, 0]
Pairs scanned: 66978
1 1
[1033864016, 1033869475, 0]
Pairs scanned: 66979
1 1
[1033864016, 1033869476, 0]
Pairs scanned: 66980
1 1
[1033864016, 1033869477, 0]
Pairs scanned: 66981
1 1
[1033864016, 1033869478, 0]
Pairs scanned: 66982
1 2
[1033864016, 1033869479, 0]
Pairs scanned: 66983
1 2
[1033864016, 1033869480, 0]
Pairs scanned: 66984
1 2
[1033864016, 1033869481, 0]
Pairs scanned: 66985
1 2
[1033864016, 1033869482, 0]
Pairs scanned: 66986

KeyboardInterrupt: ignored

##Debugging Codes 


###Checking how URLs referring to an event are found

In [ ]:
#start date as 20220315020434, end date as 20220315033434

In [59]:
dates_to_read = get_date_range()
df_mentions_agg, scanned_mention_filenames = read_mentions_file(dates_to_read)

df_export_agg, scanned_export_filenames = read_export_file(dates_to_read)

Input datetime you want to start at? (Format: YYYYmmddHHMMSS) 20220315020434
2022-03-15 02:04:34 is your start date
Input datetime you want to end at? (Format: YYYYmmddHHMMSS) 20220315033434
2022-03-15 03:34:34 is your end date
2022-03-15 03:30:00 is your new end date
2022-03-15 02:00:00 is your new start date
no file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315020000.mentions.CSV.zip .
no file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_export/20220315020000.export.CSV.zip .


In [60]:
for a in df_export_agg: 
  print(len(a))

1712
936
944
1108
1244
1013


In [28]:
print(scanned_mention_filenames)
print(len(scanned_mention_filenames))

['/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315021500.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315023000.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315024500.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315030000.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315031500.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315033000.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315034500.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315040000.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315041500.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315043000.mentions.CSV.zip', '/content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315044500.mentions.CSV.zip'

In [ ]:
#sample event ID: 1033866647

In [ ]:
# sameple event comparison: [1033864007, 1033865166, 1

In [21]:
URL_Sample_list = get_URLs_mentioning_event(1033866647, df_mentions_agg)

In [22]:
print(len(URL_Sample_list))

19


In [24]:
URL_Sample_list1 = get_URLs_mentioning_event(1033864007, df_mentions_agg)
print(URL_Sample_list1)

URL_Sample_list2 = get_URLs_mentioning_event(1033865166, df_mentions_agg)
print(URL_Sample_list2)

['https://www.proactiveinvestors.com.au/companies/news/976694/magmatic-resources-new-managing-director-starts-today-bringing-extensive-record-of-discovery-976694.html']
['http://www.msn.com/en-us/news/world/u-s-says-china-talks-on-russia-were-a-substantial-discussion/ar-AAV2aDv']


In [25]:
print(URL_Sample_list1[0] in URL_Sample_list2)

False


In [ ]:
print(URL_Sample_list)

['https://www.chron.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.thehour.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.sfgate.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.darientimes.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.seattlepi.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.nhregister.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.theridgefieldpress.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.ctpost.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php', 'https://www.ctinsider.com/news/article/Stacey-Abrams-says-she-s-more-ready-to-be-Georgia-17002287.php']


##Dividing the terminal dates into intervals (of hours/days/weeks/months) (Not Yet Used)

In [ ]:
"""def divide_daterange(datetimes_range_int, interval):
  daterange_divided = []
  scanner = 0

  def divide_daterange_hour(datetimes_range_int):
    hour = (datetimes_range_int[scanner] // 10**len(MMSS) ) % (10**len(HH))
    hourly_interval = []
    while (hour == (datetimes_range_int[scanner] // 10**len(MMSS) ) % (10**len(HH))):
      hourly_interval.append(datetimes_range_int[scanner])
      scanner += 1
    return hourly_interval 

  def divide_daterange_day(datetimes_range_int):
    day = (datetimes_range_int[scanner] // 10**len(HHMMSS) ) % (10**len(DD))
    daily_interval = []
    while (day == (datetimes_range_int[scanner] // 10**len(HHMMSS) ) % (10**len(DD))):
      daily_interval.append(datetimes_range_int[scanner])
      scanner += 1
    return daily_interval 
  
  def divide_daterange_week(datetimes_range_int):


  return daterange_divided"""

##For referencing

In [61]:
for i in range(len(df_export_agg)):
  print("Export file of " + scanned_export_filenames[i])
  with pd.option_context('display.max_rows', None,'display.max_columns', None): #'display.precision', 2,/#'display.max_colwidth', -1)
    #display(df_export_aggregate[i])
    #display(df_export_aggregate[i].sort_values('GLOBALEVENTID,INTEGER', ascending=False))
    #display(df_export_aggregate[i].sort_values('NumMentions,INTEGER', ascending=False))
    #display(df_export_aggregate[i].sort_values('DATEADDED,INTEGER', ascending=True))
    df = df_export_agg[i]
    #display(df.loc[(df['GLOBALEVENTID,INTEGER'] == 1033866648) | (df['GLOBALEVENTID,INTEGER'] == 1033866647)]) #Running for georgia governor event 
    display(df.loc[(df['Actor1CountryCode,STRING'] == "RUS") & (df['Actor2CountryCode,STRING'] == "UKR") | (df['NumMentions,INTEGER'] != df['NumArticles,INTEGER'])])
    #display(df.loc[(df['GLOBALEVENTID,INTEGER'] == 1033866326) | (df['GLOBALEVENTID,INTEGER'] == 1033864495)]) #1033866326 has a mention of grade (1,10) while that of 1033864495 has a grade of (1,80)
    #display(df.loc[(df['SOURCEURL,STRING'] == "https://www.newstimes.com/news/article/Lawmakers-to-vote-on-fix-to-increase-UC-Berkeley-17001401.php")])

Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_export/20220315021500.export.CSV.zip


,"GLOBALEVENTID,INTEGER","SQLDATE,INTEGER","MonthYear,INTEGER","Year,INTEGER","FractionDate,FLOAT","Actor1Code,STRING","Actor1Name,STRING","Actor1CountryCode,STRING","Actor1KnownGroupCode,STRING","Actor1EthnicCode,STRING","Actor1Religion1Code,STRING","Actor1Religion2Code,STRING","Actor1Type1Code,STRING","Actor1Type2Code,STRING","Actor1Type3Code,STRING","Actor2Code,STRING","Actor2Name,STRING","Actor2CountryCode,STRING","Actor2KnownGroupCode,STRING","Actor2EthnicCode,STRING","Actor2Religion1Code,STRING","Actor2Religion2Code,STRING","Actor2Type1Code,STRING","Actor2Type2Code,STRING","Actor2Type3Code,STRING","IsRootEvent,INTEGER","EventCode,STRING","EventBaseCode,STRING","EventRootCode,STRING","QuadClass,INTEGER","GoldsteinScale,FLOAT","NumMentions,INTEGER","NumSources,INTEGER","NumArticles,INTEGER","AvgTone,FLOAT","Actor1Geo_Type,INTEGER","Actor1Geo_FullName,STRING","Actor1Geo_CountryCode,STRING","Actor1Geo_ADM1Code,STRING","Actor1Geo_ADM2Code,STRING","Actor1Geo_Lat,FLOAT","Actor1Geo_Long,FLOAT","Actor1Geo_FeatureID,STRING","Actor2Geo_Type,INTEGER","Actor2Geo_FullName,STRING","Actor2Geo_CountryCode,STRING","Actor2Geo_ADM1Code,STRING","Actor2Geo_ADM2Code,STRING","Actor2Geo_Lat,FLOAT","Actor2Geo_Long,FLOAT","Actor2Geo_FeatureID,STRING","ActionGeo_Type,INTEGER","ActionGeo_FullName,STRING","ActionGeo_CountryCode,STRING","ActionGeo_ADM1Code,STRING","ActionGeo_ADM2Code,STRING","ActionGeo_Lat,FLOAT","ActionGeo_Long,FLOAT","ActionGeo_FeatureID,STRING","DATEADDED,INTEGER","SOURCEURL,STRING"
72,1033864079,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GOV,PRESIDENT,NaN,NaN,NaN,NaN,NaN,GOV,NaN,NaN,1,100,100,10,3,-5.0,12,1,6,-1.882845,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Fulani, Zamfara, Nigeria",NI,NI57,191424,12.265800,6.63738,-2029580,4,"Fulani, Zamfara, Nigeria",NI,NI57,191424,12.265800,6.63738,-2029580,20220315021500,https://www.vanguardngr.com/2022/03/worsening-...
255,1033864262,20220315,202203,2022,2022.2055,BUS,COMPANIES,NaN,NaN,NaN,NaN,NaN,BUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20,20,2,1,3.0,20,1,10,1.005747,2,"West Virginia, United States",US,USWV,NaN,38.468000,-80.96960,WV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"West Virginia, United States",US,USWV,NaN,38.468000,-80.96960,WV,20220315021500,https://www.wvgazettemail.com/news/2022-broadb...
295,1033864302,20220315,202203,2022,2022.2055,CAN,CANADA,CAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,128,128,12,3,-5.0,8,1,4,-0.845070,1,Canada,CA,CA,NaN,60.000000,-96.00000,CA,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Canada,CA,CA,NaN,60.000000,-96.00000,CA,20220315021500,https://www.chathamdailynews.ca/fp-work/unifor...
377,1033864384,20220315,202203,2022,2022.2055,COP,STATE PRISON,NaN,NaN,NaN,NaN,NaN,COP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,173,173,17,4,-5.0,5,1,4,-5.574324,3,"Johnstown, Pennsylvania, United States",US,USPA,PA075,40.278400,-76.31630,1178131,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Johnstown, Pennsylvania, United States",US,USPA,PA075,40.278400,-76.31630,1178131,20220315021500,https://www.tribdem.com/news/man-convicted-in-...
474,1033864481,20220315,202203,2022,2022.2055,EDU,SCHOOL,NaN,NaN,NaN,NaN,NaN,EDU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,14,14,1,1,0.0,15,2,12,-2.266034,3,"Washington, District of Columbia, United States",US,USDC,DC001,38.895100,-77.03640,531871,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Washington, District of Columbia, United States",US,USDC,DC001,38.895100,-77.03640,531871,20220315021500,https://www.kgw.com/article/news/education/ore...
507,1033864514,20220315,202203,2022,2022.2055,EDU,STUDENT,NaN,NaN,NaN,NaN,NaN,EDU,NaN,NaN,USA,FLORIDA,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,141,141,14,3,-6.5,13,1,9,0.086881,2,"Massachusetts, United States",US,USMA,NaN,42.237300,-71.53140,MA,2,"Florida, United States",US,USFL,NaN,27.833300,-81.71700,FL,2,"Massachusetts, United States",US,USMA,NaN,42.237300,-71.53140,MA,20220315021500,https://www.jpost.com/judaism/article-701298
565,1033864572,20220315,202203,2022,2022.205

Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_export/20220315023000.export.CSV.zip


,"GLOBALEVENTID,INTEGER","SQLDATE,INTEGER","MonthYear,INTEGER","Year,INTEGER","FractionDate,FLOAT","Actor1Code,STRING","Actor1Name,STRING","Actor1CountryCode,STRING","Actor1KnownGroupCode,STRING","Actor1EthnicCode,STRING","Actor1Religion1Code,STRING","Actor1Religion2Code,STRING","Actor1Type1Code,STRING","Actor1Type2Code,STRING","Actor1Type3Code,STRING","Actor2Code,STRING","Actor2Name,STRING","Actor2CountryCode,STRING","Actor2KnownGroupCode,STRING","Actor2EthnicCode,STRING","Actor2Religion1Code,STRING","Actor2Religion2Code,STRING","Actor2Type1Code,STRING","Actor2Type2Code,STRING","Actor2Type3Code,STRING","IsRootEvent,INTEGER","EventCode,STRING","EventBaseCode,STRING","EventRootCode,STRING","QuadClass,INTEGER","GoldsteinScale,FLOAT","NumMentions,INTEGER","NumSources,INTEGER","NumArticles,INTEGER","AvgTone,FLOAT","Actor1Geo_Type,INTEGER","Actor1Geo_FullName,STRING","Actor1Geo_CountryCode,STRING","Actor1Geo_ADM1Code,STRING","Actor1Geo_ADM2Code,STRING","Actor1Geo_Lat,FLOAT","Actor1Geo_Long,FLOAT","Actor1Geo_FeatureID,STRING","Actor2Geo_Type,INTEGER","Actor2Geo_FullName,STRING","Actor2Geo_CountryCode,STRING","Actor2Geo_ADM1Code,STRING","Actor2Geo_ADM2Code,STRING","Actor2Geo_Lat,FLOAT","Actor2Geo_Long,FLOAT","Actor2Geo_FeatureID,STRING","ActionGeo_Type,INTEGER","ActionGeo_FullName,STRING","ActionGeo_CountryCode,STRING","ActionGeo_ADM1Code,STRING","ActionGeo_ADM2Code,STRING","ActionGeo_Lat,FLOAT","ActionGeo_Long,FLOAT","ActionGeo_FeatureID,STRING","DATEADDED,INTEGER","SOURCEURL,STRING"
12,1033866263,20220213,202202,2022,2022.1178,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,173,173,17,4,-5.0,4,1,2,-6.470588,2,"Kentucky, United States",US,USKY,NaN,37.66900,-84.65140,KY,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"Kentucky, United States",US,USKY,NaN,37.669000,-84.651400,KY,20220315023000,https://www.kcra.com/article/breonna-taylor-mo...
102,1033866353,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VNM,VIETNAM,VNM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20,20,2,1,3.0,6,1,4,-4.855842,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"Vietnam, Republic Of",VM,VM,NaN,16.166667,107.833333,VM,1,"Vietnam, Republic Of",VM,VM,NaN,16.166667,107.833333,VM,20220315023000,https://dailyridge.com/2016/07/21/drunk-man-as...
143,1033866394,20220315,202203,2022,2022.2055,BUS,BUSINESS,NaN,NaN,NaN,NaN,NaN,BUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,190,190,19,4,-10.0,22,1,10,-1.612903,3,"Pinellas Park, Florida, United States",US,USFL,FL103,27.84280,-82.69950,288936,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Pinellas Park, Florida, United States",US,USFL,NaN,27.842800,-82.699500,288936,20220315023000,https://www.abcactionnews.com/news/region-pine...
144,1033866395,20220315,202203,2022,2022.2055,BUS,BUSINESS,NaN,NaN,NaN,NaN,NaN,BUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,190,190,19,4,-10.0,8,1,4,-1.612903,2,"Florida, United States",US,USFL,NaN,27.83330,-81.71700,FL,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"Florida, United States",US,USFL,NaN,27.833300,-81.717000,FL,20220315023000,https://www.abcactionnews.com/news/region-pine...
290,1033866541,20220315,202203,2022,2022.2055,EDU,STUDENT,NaN,NaN,NaN,NaN,NaN,EDU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,55,55,5,1,7.0,8,1,4,-5.083460,3,"Chatham County, North Carolina, United States",US,USNC,NC037,35.70010,-79.26640,1008544,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Chatham County, North Carolina, United States",US,USNC,NaN,35.700100,-79.266400,1008544,20220315023000,https://www.msn.com/en-us/news/us/an-apology-c...
291,1033866542,20220315,202203,2022,2022.2055,EDU,STUDENT,NaN,NaN,NaN,NaN,NaN,EDU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,55,55,5,1,7.0,12,1,6,-5.083460,3,"Raleigh, North Carolina, United States",US,USNC,NC183,35.77210,-78.63860,1024242,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Raleigh, North Carolina, United States",US,USNC,NC183,35.772100,-78.638600,1024242,20220315023000,https://www.msn.com/en-us/news/us/an-apology-c...
372,1033866623,20220315,202203,2022,2022.2055,G

Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_export/20220315024500.export.CSV.zip


...

,"GLOBALEVENTID,INTEGER","SQLDATE,INTEGER","MonthYear,INTEGER","Year,INTEGER","FractionDate,FLOAT","Actor1Code,STRING","Actor1Name,STRING","Actor1CountryCode,STRING","Actor1KnownGroupCode,STRING","Actor1EthnicCode,STRING","Actor1Religion1Code,STRING","Actor1Religion2Code,STRING","Actor1Type1Code,STRING","Actor1Type2Code,STRING","Actor1Type3Code,STRING","Actor2Code,STRING","Actor2Name,STRING","Actor2CountryCode,STRING","Actor2KnownGroupCode,STRING","Actor2EthnicCode,STRING","Actor2Religion1Code,STRING","Actor2Religion2Code,STRING","Actor2Type1Code,STRING","Actor2Type2Code,STRING","Actor2Type3Code,STRING","IsRootEvent,INTEGER","EventCode,STRING","EventBaseCode,STRING","EventRootCode,STRING","QuadClass,INTEGER","GoldsteinScale,FLOAT","NumMentions,INTEGER","NumSources,INTEGER","NumArticles,INTEGER","AvgTone,FLOAT","Actor1Geo_Type,INTEGER","Actor1Geo_FullName,STRING","Actor1Geo_CountryCode,STRING","Actor1Geo_ADM1Code,STRING","Actor1Geo_ADM2Code,STRING","Actor1Geo_Lat,FLOAT","Actor1Geo_Long,FLOAT","Actor1Geo_FeatureID,STRING","Actor2Geo_Type,INTEGER","Actor2Geo_FullName,STRING","Actor2Geo_CountryCode,STRING","Actor2Geo_ADM1Code,STRING","Actor2Geo_ADM2Code,STRING","Actor2Geo_Lat,FLOAT","Actor2Geo_Long,FLOAT","Actor2Geo_FeatureID,STRING","ActionGeo_Type,INTEGER","ActionGeo_FullName,STRING","ActionGeo_CountryCode,STRING","ActionGeo_ADM1Code,STRING","ActionGeo_ADM2Code,STRING","ActionGeo_Lat,FLOAT","ActionGeo_Long,FLOAT","ActionGeo_FeatureID,STRING","DATEADDED,INTEGER","SOURCEURL,STRING"
9,1033868656,20220213,202202,2022,2022.1178,RUSGOV,RUSSIAN,RUS,NaN,NaN,NaN,NaN,GOV,NaN,NaN,UKR,UKRAINIAN,UKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10,10,1,1,0.0,6,1,6,-9.570552,4,"Moscow, Moskva, Russia",RS,RS48,25106,55.75220,37.6156,-2960561,1,Ukraine,UP,UP,NaN,49.00000,32.0000,UP,4,"Moscow, Moskva, Russia",RS,RS48,25106,55.75220,37.615600,-2960561,20220315030000,https://www.digitaljournal.com/world/german-ww...
56,1033868703,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHN,CHINA,CHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,11,1,8,-3.030303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Guangdong, Jilin, China",CH,CH05,13176,42.76830,129.3360,10884162,4,"Guangdong, Jilin, China",CH,CH05,13176,42.76830,129.336000,10884162,20220315030000,http://en.people.cn/n3/2022/0315/c90000-997108...
57,1033868704,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHN,CHINA,CHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,20,1,8,-3.030303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Guangdong, Jilin, China",CH,CH05,13176,42.76830,129.3360,10884162,1,Hong Kong,HK,HK,NaN,22.25000,114.166667,HK,20220315030000,http://en.people.cn/n3/2022/0315/c90000-997108...
83,1033868730,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDN,INDONESIA,IDN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,12,1,10,3.365385,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Labuan Bajo, Sulawesi Tengah, Indonesia",ID,ID21,18585,-0.06320,120.0360,-2683970,4,"Labuan Bajo, Sulawesi Tengah, Indonesia",ID,ID21,18585,-0.06320,120.036000,-2683970,20220315030000,https://en.vietnamplus.vn/cambodia-announces-i...
138,1033868785,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,173,173,17,4,-5.0,10,1,4,-10.784314,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Charlestown, Rhode Island, United States",US,USRI,RI009,41.38320,-71.6417,1217684,3,"Charlestown, Rhode Island, United States",US,USRI,RI009,41.38320,-71.641700,1217684,20220315030000,https://www.thewesterlysun.com/news/police-cou...
188,1033868835,20220315,202203,2022,2022.2055,AUS,AUSTRALIA,AUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EDU,UNIVERSITY,NaN,NaN,NaN,NaN,NaN,EDU,NaN,NaN,0,90,90,9,2,-2.0,8,1,4,-4.804612,4,"Sydney, New South Wales, Australia",AS,AS02,154637,-33.88330,151.2170,-1603135,4,"Sydney, New South Wales, Australia",AS,AS02,154637,-33.88330,151.2170,-1603135,4,"Sydney, New South Wales, Australia",AS,AS02,154637,-33.88330,151.217000,-1603135,20220315030000,https://www.9new

Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_export/20220315031500.export.CSV.zip


,"GLOBALEVENTID,INTEGER","SQLDATE,INTEGER","MonthYear,INTEGER","Year,INTEGER","FractionDate,FLOAT","Actor1Code,STRING","Actor1Name,STRING","Actor1CountryCode,STRING","Actor1KnownGroupCode,STRING","Actor1EthnicCode,STRING","Actor1Religion1Code,STRING","Actor1Religion2Code,STRING","Actor1Type1Code,STRING","Actor1Type2Code,STRING","Actor1Type3Code,STRING","Actor2Code,STRING","Actor2Name,STRING","Actor2CountryCode,STRING","Actor2KnownGroupCode,STRING","Actor2EthnicCode,STRING","Actor2Religion1Code,STRING","Actor2Religion2Code,STRING","Actor2Type1Code,STRING","Actor2Type2Code,STRING","Actor2Type3Code,STRING","IsRootEvent,INTEGER","EventCode,STRING","EventBaseCode,STRING","EventRootCode,STRING","QuadClass,INTEGER","GoldsteinScale,FLOAT","NumMentions,INTEGER","NumSources,INTEGER","NumArticles,INTEGER","AvgTone,FLOAT","Actor1Geo_Type,INTEGER","Actor1Geo_FullName,STRING","Actor1Geo_CountryCode,STRING","Actor1Geo_ADM1Code,STRING","Actor1Geo_ADM2Code,STRING","Actor1Geo_Lat,FLOAT","Actor1Geo_Long,FLOAT","Actor1Geo_FeatureID,STRING","Actor2Geo_Type,INTEGER","Actor2Geo_FullName,STRING","Actor2Geo_CountryCode,STRING","Actor2Geo_ADM1Code,STRING","Actor2Geo_ADM2Code,STRING","Actor2Geo_Lat,FLOAT","Actor2Geo_Long,FLOAT","Actor2Geo_FeatureID,STRING","ActionGeo_Type,INTEGER","ActionGeo_FullName,STRING","ActionGeo_CountryCode,STRING","ActionGeo_ADM1Code,STRING","ActionGeo_ADM2Code,STRING","ActionGeo_Lat,FLOAT","ActionGeo_Long,FLOAT","ActionGeo_FeatureID,STRING","DATEADDED,INTEGER","SOURCEURL,STRING"
10,1033870217,20220308,202203,2022,2022.1863,RUS,RUSSIA,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UKRGOV,UKRAINE,UKR,NaN,NaN,NaN,NaN,GOV,NaN,NaN,0,46,46,4,1,7.0,1,1,1,0.554017,1,Russia,RS,RS,NaN,60.000000,100.0000,RS,4,"Luhansk, Luhans'ka Oblast', Ukraine",UP,UP14,25090,48.5670,39.3171,-1045160,4,"Luhansk, Luhans'ka Oblast', Ukraine",UP,UP14,25090,48.56700,39.3171,-1045160,20220315031500,https://news.antiwar.com/2022/03/14/ukraine-sa...
11,1033870218,20220308,202203,2022,2022.1863,RUS,RUSSIAN,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UKRGOV,UKRAINIAN,UKR,NaN,NaN,NaN,NaN,GOV,NaN,NaN,0,46,46,4,1,7.0,2,1,2,0.554017,1,Russia,RS,RS,NaN,60.000000,100.0000,RS,4,"Luhansk, Luhans'ka Oblast', Ukraine",UP,UP14,25090,48.5670,39.3171,-1045160,1,Russia,RS,RS,NaN,60.00000,100.0000,RS,20220315031500,https://news.antiwar.com/2022/03/14/ukraine-sa...
80,1033870287,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LAB,TRADE UNION,NaN,NaN,NaN,NaN,NaN,LAB,NaN,NaN,1,20,20,2,1,3.0,9,1,7,-2.777778,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"New Delhi, Delhi, India",IN,IN07,17911,28.6000,77.2000,-2106102,4,"New Delhi, Delhi, India",IN,IN07,17911,28.60000,77.2000,-2106102,20220315031500,https://www.tribuneindia.com/news/punjab/rift-...
109,1033870316,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UKR,UKRAINE,UKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42,42,4,1,1.9,6,1,4,-4.648589,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Odesa, Odes'ka Oblast, Ukraine",UP,UP17,28558,46.4639,30.7386,-1049092,4,"Odesa, Odes'ka Oblast, Ukraine",UP,UP17,28558,46.46390,30.7386,-1049092,20220315031500,https://bdnews24.com/world/europe/2022/03/15/t...
117,1033870324,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,36,36,3,1,4.0,10,1,6,-3.502415,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Elk City, Oklahoma, United States",US,USOK,NaN,35.4120,-99.4043,1092527,3,"Elk City, Oklahoma, United States",US,USOK,NaN,35.41200,-99.4043,1092527,20220315031500,https://www.kecofm.com/news/colorado-man-dies-...
125,1033870332,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,WASHINGTON,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,42,42,4,1,1.9,7,1,6,-3.502415,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Washington, District of Columbia, United States",US,USDC,DC001,38.8951,-77.0364,531871,2,"California, United States",US,USCA,NaN,36.17000,-119.7460,CA,20220315031500,https://www.kecofm.com/news/colorado-man-dies-...
138,1033870345,20220315,202203,2022,2022.2

Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_export/20220315033000.export.CSV.zip


,"GLOBALEVENTID,INTEGER","SQLDATE,INTEGER","MonthYear,INTEGER","Year,INTEGER","FractionDate,FLOAT","Actor1Code,STRING","Actor1Name,STRING","Actor1CountryCode,STRING","Actor1KnownGroupCode,STRING","Actor1EthnicCode,STRING","Actor1Religion1Code,STRING","Actor1Religion2Code,STRING","Actor1Type1Code,STRING","Actor1Type2Code,STRING","Actor1Type3Code,STRING","Actor2Code,STRING","Actor2Name,STRING","Actor2CountryCode,STRING","Actor2KnownGroupCode,STRING","Actor2EthnicCode,STRING","Actor2Religion1Code,STRING","Actor2Religion2Code,STRING","Actor2Type1Code,STRING","Actor2Type2Code,STRING","Actor2Type3Code,STRING","IsRootEvent,INTEGER","EventCode,STRING","EventBaseCode,STRING","EventRootCode,STRING","QuadClass,INTEGER","GoldsteinScale,FLOAT","NumMentions,INTEGER","NumSources,INTEGER","NumArticles,INTEGER","AvgTone,FLOAT","Actor1Geo_Type,INTEGER","Actor1Geo_FullName,STRING","Actor1Geo_CountryCode,STRING","Actor1Geo_ADM1Code,STRING","Actor1Geo_ADM2Code,STRING","Actor1Geo_Lat,FLOAT","Actor1Geo_Long,FLOAT","Actor1Geo_FeatureID,STRING","Actor2Geo_Type,INTEGER","Actor2Geo_FullName,STRING","Actor2Geo_CountryCode,STRING","Actor2Geo_ADM1Code,STRING","Actor2Geo_ADM2Code,STRING","Actor2Geo_Lat,FLOAT","Actor2Geo_Long,FLOAT","Actor2Geo_FeatureID,STRING","ActionGeo_Type,INTEGER","ActionGeo_FullName,STRING","ActionGeo_CountryCode,STRING","ActionGeo_ADM1Code,STRING","ActionGeo_ADM2Code,STRING","ActionGeo_Lat,FLOAT","ActionGeo_Long,FLOAT","ActionGeo_FeatureID,STRING","DATEADDED,INTEGER","SOURCEURL,STRING"
98,1033871934,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUS,RUSSIA,RUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,50,50,5,1,3.5,3,1,2,-3.705991,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Tehran, Tehran, Iran",IR,IR26,41130,35.750000,51.5148,10074674,4,"Tehran, Tehran, Iran",IR,IR26,41130,35.750000,51.5148,10074674,20220315033000,https://www.breitbart.com/politics/2022/03/14/...
103,1033871939,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,THA,THAILAND,THA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,16,1,8,0.543478,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Betong, Yala, Thailand",TH,TH70,27182,5.774340,101.0720,-3246450,4,"Betong, Yala, Thailand",TH,TH70,27182,5.774340,101.0720,-3246450,20220315033000,https://www.nationthailand.com/in-focus/40013394
116,1033871952,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,MISSOURI,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,20,1,10,1.875000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"Missouri, United States",US,USMO,NaN,38.462300,-92.3020,MO,2,"Missouri, United States",US,USMO,NaN,38.462300,-92.3020,MO,20220315033000,https://www.komu.com/news/midmissourinews/miss...
124,1033871960,20220315,202203,2022,2022.2055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,UNITED STATES,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,173,173,17,4,-5.0,6,1,4,-10.485934,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,"Montgomery County, Pennsylvania, United States",US,USPA,NaN,40.375400,-75.4999,1213680,3,"Montgomery County, Pennsylvania, United States",US,USPA,NaN,40.375400,-75.4999,1213680,20220315033000,https://www.citizensvoice.com/news/crime-emerg...
140,1033871976,20220315,202203,2022,2022.2055,AUS,MELBOURNE,AUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,180,180,18,4,-9.0,12,1,10,-2.547771,4,"Melbourne, Victoria, Australia",AS,AS07,5430,-37.816700,144.9670,-1586844,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Melbourne, Victoria, Australia",AS,AS07,5430,-37.816700,144.9670,-1586844,20220315033000,https://www.dailymail.co.uk/news/article-10613...
161,1033871997,20220315,202203,2022,2022.2055,BUS,AIRLINE,NaN,NaN,NaN,NaN,NaN,BUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,111,111,11,3,-2.0,8,1,4,0.595238,4,"Delhi, Delhi, India",IN,IN07,17911,28.666700,77.2167,-2094230,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,"Delhi, Delhi, India",IN,IN07,17911,28.666700,77.2167,-2094230,20220315033000,https://www.afaqs.com/news/mktg/flight-booking...
162,1033871998,20220315,202203,2022,2022.2055,BUS,AIRLINE,NaN,NaN,NaN,NaN,NaN

In [ ]:
#Website of global event ID 1033866648	
df_sample = df_export_aggregate[0]
df_sample_2 = df_export_aggregate[1]
"""website_1033866647 = df_sample.iloc[396]['SOURCEURL,STRING']
website_1033866648 = df_sample.iloc[397]['SOURCEURL,STRING']
print(website_1033866647)
print(website_1033866648)"""

# getting the website of a low confidence event (1033866326) and a high confidence event (1033864495)
website_1033864495 = df_sample.iloc[488]['SOURCEURL,STRING'] #Lawmakers event 
website_1033866326 = df_sample_2.iloc[75]['SOURCEURL,STRING'] #Ukraine event

print(website_1033866326)
print(website_1033864495)

https://www.darientimes.com/news/article/In-Ukraine-female-war-reporters-build-on-legacy-17000778.php
https://www.newstimes.com/news/article/Lawmakers-to-vote-on-fix-to-increase-UC-Berkeley-17001401.php


testing the reading of the mentions file

In [76]:
for i in range(len(df_mentions_agg)):
  print("Export file of " + scanned_mention_filenames[i])
  with pd.option_context('display.max_rows', None,'display.max_columns', None): #'display.precision', 2,/#'display.max_colwidth', -1)
    #display(df_mentions_aggregate[i])
    #display(df_mentions_aggregate[i].sort_values('*GlobalEventID. (integer*', ascending=False))
    df = df_mentions_agg[i]
    #display(df.loc[(df['*GlobalEventID. (integer*'] == 1033866647) | (df['*GlobalEventID. (integer*'] == 1033866648)])
    display(df.loc[df['* MentionSourceName. (integer*'] == "thehour.com"])
    #display(df.loc[(df['*GlobalEventID. (integer*'] == 1033864495)])

Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315021500.mentions.CSV.zip


,*GlobalEventID. (integer*,* EventTimeDate. (integer*,* MentionTimeDate. (integer*,* MentionType. (integer*,* MentionSourceName. (integer*,* MentionIdentifier. (integer*,* SentenceID. (integer*,* Actor1CharOffset. (integer*,* Actor2CharOffset. (integer*,* ActionCharOffset. (integer*,* InRawText. (integer*,* Confidence. (integer*,* MentionDocLen. (integer*,* MentionDocTone. (integer*,* MentionDocTranslationInfo. (string*,* Extras. (string*


Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315023000.mentions.CSV.zip


,*GlobalEventID. (integer*,* EventTimeDate. (integer*,* MentionTimeDate. (integer*,* MentionType. (integer*,* MentionSourceName. (integer*,* MentionIdentifier. (integer*,* SentenceID. (integer*,* Actor1CharOffset. (integer*,* Actor2CharOffset. (integer*,* ActionCharOffset. (integer*,* InRawText. (integer*,* Confidence. (integer*,* MentionDocLen. (integer*,* MentionDocTone. (integer*,* MentionDocTranslationInfo. (string*,* Extras. (string*
235,1033866326,20220315023000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/In-Ukrain...,5,-1,985,973,1,10,8965,-2.546917,NaN,NaN
240,1033861643,20220315020000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/In-Ukrain...,5,-1,985,973,0,40,8965,-2.546917,NaN,NaN
245,1033846500,20220315003000,20220315023000,1,thehour.com,https://www.thehour.com/westport/article/Westp...,2,-1,1442,1382,1,60,3294,1.592920,NaN,NaN
246,1033846501,20220315003000,20220315023000,1,thehour.com,https://www.thehour.com/westport/article/Westp...,2,-1,1432,1372,0,20,3294,1.592920,NaN,NaN
247,1033846144,20220315001500,20220315023000,1,thehour.com,https://www.thehour.com/westport/article/Westp...,2,-1,1453,1393,0,20,3294,1.592920,NaN,NaN
297,1033846531,20220315003000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/Cargo-shi...,1,-1,356,329,0,60,591,-4.672897,NaN,NaN
311,1033846543,20220315003000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/Cargo-shi...,1,-1,350,329,1,40,591,-4.672897,NaN,NaN
464,1033861757,20220315020000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/In-Ukrain...,20,6851,6816,6842,0,20,8965,-2.546917,NaN,NaN
468,1033861758,20220315020000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/In-Ukrain...,20,6838,6800,6828,1,80,8965,-2.546917,NaN,NaN
734,1033839944,20220315000000,20220315023000,1,thehour.com,https://www.thehour.com/news/article/Another-p...,5,1286,-1,1270,1,50,2069,-4.049844,NaN,NaN


Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315024500.mentions.CSV.zip


...

,*GlobalEventID. (integer*,* EventTimeDate. (integer*,* MentionTimeDate. (integer*,* MentionType. (integer*,* MentionSourceName. (integer*,* MentionIdentifier. (integer*,* SentenceID. (integer*,* Actor1CharOffset. (integer*,* Actor2CharOffset. (integer*,* ActionCharOffset. (integer*,* InRawText. (integer*,* Confidence. (integer*,* MentionDocLen. (integer*,* MentionDocTone. (integer*,* MentionDocTranslationInfo. (string*,* Extras. (string*


Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315031500.mentions.CSV.zip


,*GlobalEventID. (integer*,* EventTimeDate. (integer*,* MentionTimeDate. (integer*,* MentionType. (integer*,* MentionSourceName. (integer*,* MentionIdentifier. (integer*,* SentenceID. (integer*,* Actor1CharOffset. (integer*,* Actor2CharOffset. (integer*,* ActionCharOffset. (integer*,* InRawText. (integer*,* Confidence. (integer*,* MentionDocLen. (integer*,* MentionDocTone. (integer*,* MentionDocTranslationInfo. (string*,* Extras. (string*


Export file of /content/drive/MyDrive/GDELT_Raw_data/GDELT_v2_mentions/20220315033000.mentions.CSV.zip


,*GlobalEventID. (integer*,* EventTimeDate. (integer*,* MentionTimeDate. (integer*,* MentionType. (integer*,* MentionSourceName. (integer*,* MentionIdentifier. (integer*,* SentenceID. (integer*,* Actor1CharOffset. (integer*,* Actor2CharOffset. (integer*,* ActionCharOffset. (integer*,* InRawText. (integer*,* Confidence. (integer*,* MentionDocLen. (integer*,* MentionDocTone. (integer*,* MentionDocTranslationInfo. (string*,* Extras. (string*


In [ ]:
print(df_men_sample.iloc[1077]['* MentionIdentifier. (integer*'])
df_men_sample = df_mentions_aggregate[0]
print(df_men_sample.iloc[1345]['* MentionIdentifier. (integer*'])

https://www.thehour.com/news/article/Lawmakers-to-vote-on-fix-to-increase-UC-Berkeley-17001401.php
https://www.newstimes.com/news/article/Lawmakers-to-vote-on-fix-to-increase-UC-Berkeley-17001401.php


#Misc Testing

In [ ]:

a = [['a','b'], ['c']]
print(list(itertools.chain.from_iterable(a)))

['a', 'b', 'c']


In [1]:
1 in [1,2,3]

True

In [62]:
a = np.array([[1,2,3],[4,5,6]])

In [63]:
a.transpose()

array([[1, 4],
       [2, 5],
       [3, 6]])

In [75]:
b = itertools.combinations(a.transpose(), 2)

for combination in itertools.combinations(a.transpose(),2):
  print(list(combination)[0])

[1 4]
[1 4]
[2 5]


In [185]:
np.array([1,2]) in np.array([[1,2],[3,4]])

True